# Advanced Lane Finding - Project 4 of Udacity's Self-Driving Car Nanodegree

1. Camera Calibration
2. Distortion correction
3. Image inspection
4. Perspective correction

## 1. Camera Calibration

In [ ]:
%run AdvLaneCameraCalibrator.py

In [ ]:
camdata = open('camera_calib.pkl', 'rb')
mtx = pickle.load(camdata)
dist = pickle.load(camdata)
output.close()

print(mtx)
print(dist)

## 2. Distortion Correction

In [ ]:
images = glob.glob('camera_cal/calibration*.jpg')

fig = plt.figure(figsize=(16,32))
index = 0

# Step through the list and search for chessboard corners
for fname in images:
    img = cv2.imread(fname)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.undistort(img, mtx, dist, None, mtx)
    sp = fig.add_subplot(row_count, col_count, index+1)
    plt.imshow(img)
    index += 1

plt.show()

## 3. Example image inspection

In [ ]:
images = ['test_images/test1.jpg', 'test_images/test2.jpg', 'test_images/test3.jpg',
          'test_images/test4.jpg', 'test_images/test5.jpg', 'test_images/test6.jpg',
          'test_images/straight_lines1.jpg', 'test_images/straight_lines2.jpg']

example_images = images

col_count = 2
row_count = 8
fig = plt.figure(figsize=(20,60))

index = 0

example_image = None

# Step through the list and search for chessboard corners
for fname in example_images:
    img = cv2.imread(fname)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    sp = fig.add_subplot(row_count, col_count, index*2+1)
    plt.imshow(img)
    img = cv2.undistort(img, mtx, dist, None, mtx)
    sp = fig.add_subplot(row_count, col_count, index*2+2)
    plt.imshow(img)
    index += 1

plt.show()

# 4. Perspective correction

In [ ]:
example_image = cv2.imread(example_images[6])
example_image = cv2.cvtColor(example_image, cv2.COLOR_BGR2RGB)
example_image = cv2.undistort(example_image, mtx, dist, None, mtx)

img_size = (example_image.shape[1],example_image.shape[0])

# defines the perspective of the camera image by defining points near the front of the car and close the the
# center of the image
relation_factor = 13.5/1.92
front_perspective_div = 3.5
back_perspective_div = front_perspective_div*relation_factor
front_perspective_y_perc = 0.92
back_perspective_y_perc = 0.63

# trapez points in order bottom left, top left, top right, bottom right (front, back, back, front)
src = [(int(img_size[0]/2-img_size[0]/front_perspective_div), int(img_size[1]*front_perspective_y_perc)), 
       (int(img_size[0]/2-img_size[0]/back_perspective_div), int(img_size[1]*back_perspective_y_perc)), 
       (int(img_size[0]/2+img_size[0]/back_perspective_div), int(img_size[1]*back_perspective_y_perc)),
       (int(img_size[0]/2+img_size[0]/front_perspective_div), int(img_size[1]*front_perspective_y_perc))]

# paint trapez into the image
image_copy = np.copy(example_image)

for index in range(4):
    pa = src[index]
    pb = src[(index+1)%4]
    cv2.line(image_copy, pa, pb, (255,0,0), 4)

fig = plt.figure(figsize=(20,20))
plt.imshow(image_copy)

margin_factor = 6

dst = [(img_size[0]//margin_factor, img_size[1]), 
       (img_size[0]//margin_factor, 0), 
       (img_size[0]-img_size[0]//margin_factor, 0), 
       (img_size[0]-img_size[0]//margin_factor, img_size[1]), 
       ]

src = np.array(src, dtype=np.float32)
dst = np.array(dst, dtype=np.float32)

tmx = cv2.getPerspectiveTransform(np.array(src), np.array(dst))

plt.show()

width = image_copy.shape[1]
height = image_copy.shape[0]

warped = cv2.warpPerspective(example_image, tmx, (width, height))

fig = plt.figure(figsize=(20,20))
plt.imshow(warped)

In [ ]:
project_video = "project_video.mp4"

HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(project_video))

### Video of the project video after distortion and perspective correction

In [ ]:
def process_image(image):
    undistorted = cv2.undistort(image, mtx, dist, None, mtx)
    warped = cv2.warpPerspective(undistorted, tmx, (width, height))
    return warped
    
from_above_video = 'test_videos_output/from_above.mp4'

white_output = from_above_video
clip1 = VideoFileClip(project_video)
white_clip = clip1.fl_image(process_image)
%time white_clip.write_videofile(white_output, audio=False)

HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(from_above_video))

## Highlighting lanes

In [ ]:
col_count = 2
row_count = 8
fig = plt.figure(figsize=(20,60))
index = 0

# Step through the list and search for chessboard corners
for fname in example_images:
    img = cv2.imread(fname)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    sp = fig.add_subplot(row_count, col_count, index+1)
    img = cv2.undistort(img, mtx, dist, None, mtx)
    plt.title(ntpath.basename(fname))
    warped = cv2.warpPerspective(img, tmx, (width, height))
    plt.imshow(warped, 'gray')
    index += 1

plt.show()

In [ ]:
def hls_threshold_mask(image, thresh=(95,255)):
    hls = cv2.cvtColor(image, cv2.COLOR_RGB2HLS)
    
    s = hls[:,:,2]
    
    binary_output = np.zeros_like(s)
    binary_output[(s>=thresh[0]) & (s<=thresh[1])] = 1

    return binary_output

def sobel_mag_mask(image, thresh=(10,255), kernel_size=3):
    image = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    sobel_x = cv2.Sobel(image, cv2.CV_64F, 1, 0, ksize=kernel_size)
    sobel_y = cv2.Sobel(image, cv2.CV_64F, 0, 1, ksize=kernel_size)
    abs_sobelxy = np.sqrt(sobel_x**2 + sobel_y**2)
    # 4) Scale to 8-bit (0 - 255) and convert to type = np.uint8
    eight_bit = np.uint8(255*abs_sobelxy/np.max(abs_sobelxy))
    # 5) Create a binary mask where mag thresholds are met
    # 6) Return this mask as your binary_output image
    binary_output = np.zeros_like(abs_sobelxy)
    binary_output[(eight_bit >= thresh[0]) & (eight_bit <= thresh[1])] = 1    

    return binary_output

def sobel_dir_mask(image, thresh=(0.7, 1.3), kernel_size=13):
    image = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    sobel_x = cv2.Sobel(image, cv2.CV_64F, 1, 0, ksize=kernel_size)
    sobel_y = cv2.Sobel(image, cv2.CV_64F, 0, 1, ksize=kernel_size)
    abs_x = np.abs(sobel_x)
    abs_y = np.abs(sobel_y)
    # 4) Use np.arctan2(abs_sobely, abs_sobelx) to calculate the direction of the gradient 
    sob_dir = np.arctan2(abs_y, abs_x)
    # 5) Create a binary mask where direction thresholds are met    
    # 6) Return this mask as your binary_output image
    binary_output = np.zeros_like(sob_dir)
    binary_output[(sob_dir>=thresh[0]) & (sob_dir<=thresh[1])] = 1
    
    return binary_output

def create_binary_mask(image):
    hls_thresh = hls_threshold_mask(image)
    sobel_mag = sobel_mag_mask(image)
    sobel_dir = sobel_dir_mask(image)
    
    binary_output = np.zeros_like(sobel_dir)
    binary_output[(hls_thresh==1)] = 1.0
#    binary_output[(hls_thresh==1) | ((sobel_mag==1) & (sobel_dir==1))] = 1
    ## binary_output[(hls_thresh==1)] = 1
    
    return binary_output

col_count = 2
row_count = 8
fig = plt.figure(figsize=(20,60))
index = 0

# Step through the list and search for chessboard corners
for fname in example_images:
    img = cv2.imread(fname)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    sp = fig.add_subplot(row_count, col_count, index+1)
    img = cv2.undistort(img, mtx, dist, None, mtx)
    # img = sobel_mag_mask(img)
    img = create_binary_mask(img)
    plt.title(ntpath.basename(fname))
    warped = cv2.warpPerspective(img, tmx, (width, height))
    plt.imshow(warped, 'gray')
    index += 1

plt.show()

In [ ]:
col_count = 2
row_count = 8
fig = plt.figure(figsize=(20,60))
index = 0

# Step through the list and search for chessboard corners
for fname in example_images:
    img = cv2.imread(fname)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    sp = fig.add_subplot(row_count, col_count, index+1)
    img = cv2.undistort(img, mtx, dist, None, mtx)
    # img = sobel_mag_mask(img)
    img = create_binary_mask(img)
    plt.title(ntpath.basename(fname))
    warped = cv2.warpPerspective(img, tmx, (width, height))
    histogram = np.sum(warped[img.shape[0]//2:,:], axis=0)
    plt.plot(histogram)
    index += 1

plt.show()

In [ ]:


for cur_fn in example_images:
    img = cv2.imread(cur_fn)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    lf = lane_finder()
    out_img = lf.find_lanes_using_window(img)
    
    fig = plt.figure(figsize=(12,8))
    plt.imshow(out_img)
#    plt.plot(left_fitx, ploty, color='yellow')
#    plt.plot(right_fitx, ploty, color='yellow')
    plt.xlim(0, 1280)
    plt.ylim(720, 0)
    plt.show()    


In [ ]:
lf = lane_finder()

def process_image(image):
    warped = lf.find_lanes_using_window(image)
    return warped
    
find_lanes_raw = 'test_videos_output/find_lanes_raw.mp4'

white_output = find_lanes_raw
clip1 = VideoFileClip(project_video)
white_clip = clip1.fl_image(process_image)
%time white_clip.write_videofile(white_output, audio=False)

HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(find_lanes_raw))